In [1]:
pip install capytaine   

You should consider upgrading via the 'c:\Users\akash\AppData\Local\Programs\Python\Python39\python.exe -m pip install --upgrade pip' command.


In [2]:
import capytaine as cpt
import logging
import vtk
import numpy as np 
import xarray as xr
logging.basicConfig(level=logging.INFO)

In [3]:
body = cpt.FloatingBody.from_file('buoy6.dat')
body.center_of_mass = (0,0,0)
body.keep_immersed_part()
body.add_all_rigid_body_dofs()

INFO:capytaine.bodies.bodies:New floating body: buoy6.dat.
INFO:capytaine.bodies.bodies:Clipping buoy6.dat with respect to Plane(normal=[0. 0. 1.], point=[0. 0. 0.])
INFO:capytaine.meshes.clipper:Clipping buoy6.dat_mesh by Plane(normal=[0. 0. 1.], point=[0. 0. 0.]): no action.
INFO:capytaine.bodies.bodies:The rotation dof Roll has been initialized around the point: buoy6.dat.center_of_mass = (0, 0, 0)
INFO:capytaine.bodies.bodies:The rotation dof Pitch has been initialized around the point: buoy6.dat.center_of_mass = (0, 0, 0)
INFO:capytaine.bodies.bodies:The rotation dof Yaw has been initialized around the point: buoy6.dat.center_of_mass = (0, 0, 0)


In [4]:
body.show()

In [10]:
freq = np.linspace(0.1,1,10)
directions = np.linspace(0,np.pi/2,2)

test_matrix = xr.Dataset(coords={
    'omega': freq,
    'wave_direction': directions,
    'radiating_dof': list(body.dofs),
    'water_depth': [np.infty],
    })
'''
problems = [cpt.RadiationProblem(body=body,
                              radiating_dof=dof,
                              omega=w,
                              sea_bottom=-np.infty,
                              g=9.81,
                              rho=1025.0)
                              for dof in body.dofs for w in freq]
problems += [cpt.DiffractionProblem(body=body,
                                omega=w,
                                wave_direction=heading,
                                sea_bottom=-np.infty,
                                g=9.81,
                                rho=1025.0)
                                for w in freq for heading in directions]
'''

'\nproblems = [cpt.RadiationProblem(body=body,\n                              radiating_dof=dof,\n                              omega=w,\n                              sea_bottom=-np.infty,\n                              g=9.81,\n                              rho=1025.0)\n                              for dof in body.dofs for w in freq]\nproblems += [cpt.DiffractionProblem(body=body,\n                                omega=w,\n                                wave_direction=heading,\n                                sea_bottom=-np.infty,\n                                g=9.81,\n                                rho=1025.0)\n                                for w in freq for heading in directions]\n'

In [11]:
solver = cpt.BEMSolver()
dataset = solver.fill_dataset(test_matrix, [body], hydrostatics=False)

c:\Users\akash\AppData\Local\Programs\Python\Python39\lib\site-packages\capytaine\io\xarray.py:68: FutureWarning: The return type of `Dataset.dims` will be changed to return a set of dimension names in future, in order to be more consistent with `DataArray.dims`. To access a mapping from dimension names to lengths, please use `Dataset.sizes`.
  keys_in_dataset = set(dataset.dims.keys())
INFO:capytaine.bem.solver:Solve DiffractionProblem(body=buoy6.dat, omega=0.100, water_depth=inf, wave_direction=0.000).
INFO:capytaine.bem.solver:Solve DiffractionProblem(body=buoy6.dat, omega=0.100, water_depth=inf, wave_direction=1.571).
INFO:capytaine.bem.solver:Solve RadiationProblem(body=buoy6.dat, omega=0.100, water_depth=inf, radiating_dof='Surge').
INFO:capytaine.bem.solver:Solve RadiationProblem(body=buoy6.dat, omega=0.100, water_depth=inf, radiating_dof='Sway').
INFO:capytaine.bem.solver:Solve RadiationProblem(body=buoy6.dat, omega=0.100, water_depth=inf, radiating_dof='Heave').
INFO:capytaine

In [12]:
results = [solver.solve(problem, keep_details=True) for problem in sorted(problems)]
capyData = cpt.assemble_dataset(results, hydrostatics=False)

INFO:capytaine.bem.solver:Solve RadiationProblem(body=buoy6.dat, omega=0.100, water_depth=inf, radiating_dof='Surge', rho=1025.0).
INFO:capytaine.bem.solver:Solve RadiationProblem(body=buoy6.dat, omega=0.100, water_depth=inf, radiating_dof='Sway', rho=1025.0).
INFO:capytaine.bem.solver:Solve RadiationProblem(body=buoy6.dat, omega=0.100, water_depth=inf, radiating_dof='Heave', rho=1025.0).
INFO:capytaine.bem.solver:Solve RadiationProblem(body=buoy6.dat, omega=0.100, water_depth=inf, radiating_dof='Roll', rho=1025.0).
INFO:capytaine.bem.solver:Solve RadiationProblem(body=buoy6.dat, omega=0.100, water_depth=inf, radiating_dof='Pitch', rho=1025.0).
INFO:capytaine.bem.solver:Solve RadiationProblem(body=buoy6.dat, omega=0.100, water_depth=inf, radiating_dof='Yaw', rho=1025.0).
INFO:capytaine.bem.solver:Solve DiffractionProblem(body=buoy6.dat, omega=0.100, water_depth=inf, wave_direction=0.000, rho=1025.0).
INFO:capytaine.bem.solver:Solve DiffractionProblem(body=buoy6.dat, omega=0.100, water_

In [13]:
ncFName = 'demo.nc'
cpt.io.xarray.separate_complex_values(capyData).to_netcdf(ncFName)

In [14]:
hydrostatics = body.compute_hydrostatics(rho = 1025.0)
from capytaine.io.legacy import export_hydrostatics
export_hydrostatics('static.dat', body)

INFO:capytaine.bodies.bodies:Clipping buoy6.dat with respect to Plane(normal=[0. 0. 1.], point=[0. 0. 0.])
INFO:capytaine.meshes.clipper:Clipping buoy6.dat_mesh by Plane(normal=[0. 0. 1.], point=[0. 0. 0.]): no action.


             You might be overwriting existing files!
